<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-the-data" data-toc-modified-id="Importing-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing the data</a></span><ul class="toc-item"><li><span><a href="#Preprocessing-the-file" data-toc-modified-id="Preprocessing-the-file-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preprocessing the file</a></span></li></ul></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Preparation</a></span><ul class="toc-item"><li><span><a href="#Focusing-on-&quot;production&quot;-partition-only" data-toc-modified-id="Focusing-on-&quot;production&quot;-partition-only-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Focusing on "production" partition only</a></span></li><li><span><a href="#Converting-Job-times-to-total-seconds" data-toc-modified-id="Converting-Job-times-to-total-seconds-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Converting Job times to total seconds</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd

# For preprocessing
import csv
from os import path
from pathlib import Path

# Set the project root folder
root_dir = Path().absolute() # wherever this notebook file is located

## Importing the data

This import has an error so we will wrap it in a Try-Except block first.

In [2]:
try:
    data = pd.read_csv('../data/accre-jobs-2020.csv')
except Exception as error:
    print(error)

Error tokenizing data. C error: Expected 13 fields in line 3461, saw 15



Looks like the error is on line index 3461 (This is 1-based so with our 0-based indexing, that would be 3460.)

Looking into the file directly, it appears that the import creates some errors because some of the data in that line are formatted as `[1,2,3,4]`. The commas here are tricking the CSV parser in thinking that they should be multiple columns instead of one single column. Let's fix that!

If we put that last column within quotes as `"[1,2,3,4]"`, the csv parser would be able to interpret it as one column.

### Preprocessing the file

Here, we read the original file, go through each line, and up to the 13th column, we keep them as is. Beyond the 13th column, we wrap all the values within quotes. We then write this into a new file that we will use as our dataset with Pandas.

**This line might take a while to run on the first time!**

In [3]:
try:
    # The data might have already been pre-processed previoulsy
    # If not, this line would generate a "No such file" error so we would fallback to the exception below
    data = pd.read_csv('../data/accre-jobs-2020-processed.csv')
    print('Reading from previously processed file...')
    display(data.head())
    
except FileNotFoundError:
    
    print("Preprocessed file not found. Creating a new one. Please wait...")
    
    # Open 2 files: One is the source and one is the destination
    with open(path.join(root_dir, '..', 'data', 'accre-jobs-2020.csv'), mode='r') as source,\
    open(path.join(root_dir, '..', 'data', 'accre-jobs-2020-processed.csv'), mode='w', newline='') as destination:

        # Set file reader and file writer on the source and destination
        reader = csv.reader(source)
        writer = csv.writer(destination)

        # Go through each line in the source
        for line in reader:
            # Create a new matching line with the last column within quotes
            # This will make it into a single column if there are more commas
            newline = line[:12] + [",".join(line[12:])]
            # Write to the destination file
            writer.writerow(newline)

    # Try importing as normal again on the newly created destination file
    data = pd.read_csv('../data/accre-jobs-2020-processed.csv')
    display(data.head())

except Error as error:
    
    print("Something went wrong:", error)

Reading from previously processed file...


,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


Now, we can work on that destination file instead

In [4]:
# Try importing as normal again on the newly created destination file
# data = pd.read_csv('../data/accre-jobs-2020-processed.csv')
# data.head()

Let's look at line 3461 (Index 3460) to confirm.

In [5]:
# Check the previously problematic line
data.iloc[[3460]]

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
3460,17050901_91,winged,lavonda,4096Mn,669.61M,12:00:00,00:06:05,4,1,production,0:0,COMPLETED,"cn[449,463,911,913]"


Yep, looks like it is imported normally. We are good to go!

## Data Preparation

First, let's look at the quick summary

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3816290 entries, 0 to 3816289
Data columns (total 13 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   JOBID      object
 1   ACCOUNT    object
 2   USER       object
 3   REQMEM     object
 4   USEDMEM    object
 5   REQTIME    object
 6   USEDTIME   object
 7   NODES      int64 
 8   CPUS       int64 
 9   PARTITION  object
 10  EXITCODE   object
 11  STATE      object
 12  NODELIST   object
dtypes: int64(2), object(11)
memory usage: 378.5+ MB


### Focusing on "production" partition only

We really only want to look at the "production" partition so rows with other partitions should be removed.

In [7]:
data_prod = data.loc[data["PARTITION"] == "production"]
data_prod.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


In [8]:
data_prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3311788 entries, 0 to 3816289
Data columns (total 13 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   JOBID      object
 1   ACCOUNT    object
 2   USER       object
 3   REQMEM     object
 4   USEDMEM    object
 5   REQTIME    object
 6   USEDTIME   object
 7   NODES      int64 
 8   CPUS       int64 
 9   PARTITION  object
 10  EXITCODE   object
 11  STATE      object
 12  NODELIST   object
dtypes: int64(2), object(11)
memory usage: 353.7+ MB


**How many are production node? Non-production nodes?**

In [9]:
data["PARTITION"].value_counts()

production              3311788
nogpfs                   327652
sam                       79151
pascal                    48004
turing                    39406
debug                      6738
maxwell                    3348
cgw-capra1                   83
cgw-dougherty1               72
cgw-cqs1                     34
cgw-cqs3                      4
cgw-vm-qa-flatearth1          4
cgw-rocksteady                2
devel                         2
cgw-tbi01                     2
Name: PARTITION, dtype: int64

### Converting Job times to total seconds

Job time is in a format of either `d-hh:mm:ss` or `hh:mm:ss`, it needs to be converted to total seconds

In [10]:
def convert_time_to_seconds(time_str):
    """Convert the given time string d-hh:mm:ss or hh:mm:ss to total seconds."""
    
    # Account for possible errors
    try:
        
        # Initialize all time components to 0
        days = hours = minutes = seconds = 0

        # Split the time string at the dash mark
        time_dash_split = time_str.split("-") # => Either ["d", "hh:mm:ss"] or ["hh:mm:ss"]

        # Assign each time element to variables
        if len(time_dash_split) > 1: 
            # Remove the first element from the list and convert to integer and add to days
            days += int(time_dash_split.pop(0))

        # If here, len(time_dash_split) == 1
        # Split the time at the colon marks
        time_colon_split = time_dash_split[0].split(":") # => ["hh", "mm", "ss"]
        seconds += int(time_colon_split.pop())
        minutes += int(time_colon_split.pop())
        hours += int(time_colon_split.pop())

        # Now combine everything together
        return (((((days * 24) + hours) * 60) + minutes) * 60) + seconds
        
    except AttributeError as error:
        # Skip everything if the column is already in seconds (assuming one single digit)
        return time_str

Now, we can convert the time columns to seconds

In [11]:
new_reqtime = data_prod["REQTIME"].map(convert_time_to_seconds)
new_usedtime = data_prod["USEDTIME"].map(convert_time_to_seconds)

data_prod = data_prod.assign(REQTIME = new_reqtime)
data_prod = data_prod.assign(USEDTIME = new_usedtime)

data_prod.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,1188000,1188028,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,1188000,1090256,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,1188000,1188020,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,1188000,1050632,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,1188000,1188003,1,24,production,0:0,COMPLETED,cn1440
